In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

<ipython-input-4-b11c832c5082>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = pd.DatetimeIndex(train['일자']).week
<ipython-input-4-b11c832c5082>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = pd.DatetimeIndex(test['일자']).week


In [5]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [6]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [7]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

## 공휴일 변수 생성

In [8]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [9]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

In [10]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [11]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 16)
(50, 16)


In [12]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)


In [13]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


In [14]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [15]:
from automl_alex import LightGBMRegressor, CatBoostRegressor, AutoMLRegressor
from sklearn.metrics import mean_absolute_error as MAE
import sklearn

In [16]:
lunch_model = AutoMLRegressor(random_state=42, metric=MAE)

lunch_model.fit(lunch_train, y_lunch,
                verbose=3,
                folds=12,
                opt_lvl=5,
                early_stoping=120,
                auto_parameters=False,
                timeout=1100
               )

15:21:09 | > Start Fit Base Model
15:21:10 | ##################################################
15:21:10 | > Start Fit Models 2
15:21:10 | ##################################################
15:21:10 | ##################################################
15:21:11 | > Step 1: calc parameters and pruned score: get test 10 trials
C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` i

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Optimize: : 1it [00:06,  6.80s/it, | Model: LightGBM | OptScore: 107.1606 | Best mean_absolute_error: 68.7503 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution wi

Optimize: : 10it [01:01,  4.27s/it, | Model: LightGBM | OptScore: 75.2383 | Best mean_absolute_error: 67.1373 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
Optimize: : 11it [01:07,  4.80s/it, | Model: LightGBM | OptScore: 67.8451 | Best mean_absolute_error: 67.1373 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in

Optimize: : 23it [02:11,  6.55s/it, | Model: LightGBM | OptScore: 66.1953 | Best mean_absolute_error: 66.0397 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
Optimize: : 24it [02:16,  5.86s/it, | Model: LightGBM | OptScore: 67.5828 | Best mean_absolute_error: 66.0397 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Optimize: : 35it [04:24,  9.87s/it, | Model: LightGBM | OptScore: 68.4128 | Best mean_absolute_error: 66.0397 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution wi

Optimize: : 47it [07:59, 12.94s/it, | Model: LightGBM | OptScore: 66.5972 | Best mean_absolute_error: 66.0397 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
Optimize: : 48it [08:08, 11.82s/it, | Model: LightGBM | OptScore: 66.4812 | Best mean_absolute_error: 66.0397 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in

Optimize: : 61it [09:48,  9.15s/it, | Model: LightGBM | OptScore: 65.1325 | Best mean_absolute_error: 64.5471 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
Optimize: : 62it [10:02, 10.51s/it, | Model: LightGBM | OptScore: 68.7107 | Best mean_absolute_error: 64.5471 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in

Optimize: : 73it [12:37, 13.65s/it, | Model: LightGBM | OptScore: 65.4009 | Best mean_absolute_error: 64.5471 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
Optimize: : 74it [12:40, 10.53s/it, | Model: LightGBM | OptScore: 73.6381 | Best mean_absolute_error: 64.5471 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in

In [17]:
predicts_Auto_lunch = lunch_model.predict(lunch_test)

In [18]:
dinner_model = AutoMLRegressor(random_state=42, metric=MAE)

dinner_model.fit(dinner_train, y_dinner,         
                 verbose=3,
                 folds=12,
                 opt_lvl=5,
                 early_stoping=120,
                 auto_parameters=False,
                 timeout=1100
                )

15:40:34 | > Start Fit Base Model
15:40:36 | ##################################################
15:40:36 | > Start Fit Models 2
15:40:36 | ##################################################
15:40:36 | ##################################################
15:40:36 | > Step 1: calc parameters and pruned score: get test 10 trials
C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` i

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Optimize: : 1it [00:06,  6.99s/it, | Model: LightGBM | OptScore: 90.714 | Best mean_absolute_error: 62.3574 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Optimize: : 7it [02:01, 23.32s/it, | Model: LightGBM | OptScore: 62.465 | Best mean_absolute_error: 62.3574 ] C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call a

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Optimize: : 17it [03:53, 11.21s/it, | Model: LightGBM | OptScore: 107.9608 | Best mean_absolute_error: 62.3574 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of arg

Optimize: : 23it [06:14, 23.73s/it, | Model: LightGBM | OptScore: 62.8049 | Best mean_absolute_error: 62.1299 ] C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'low': 0.01, 'high': 0.3}
  warnings.warn(
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of arg

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Optimize: : 31it [10:14, 37.15s/it, | Model: LightGBM | OptScore: 59.6716 | Best mean_absolute_error: 59.6484 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution wi

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Optimize: : 37it [12:54, 25.95s/it, | Model: LightGBM | OptScore: 58.3782 | Best mean_absolute_error: 58.769 ]C:\anaconda3\envs\machinelearning\lib\site-packages\optuna\trial\_trial.py:769: RuntimeWarning: Inconsistent parameter values for distribution with name "lgbm_learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more then once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call a

In [19]:
lunch_train.columns, lunch_test.columns, 

(Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
        '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
       dtype='object'),
 Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
        '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
       dtype='object'))

In [20]:
predicts_Auto_dinner = dinner_model.predict(dinner_test)

In [21]:
submission = pd.read_csv('../data/sample_submission.csv')

In [22]:
submission.iloc[:,1] = predicts_Auto_lunch
submission.iloc[:,2] = predicts_Auto_dinner
submission.head()

,일자,중식계,석식계
0,2021-01-27,987.876584,309.995352
1,2021-01-28,936.707541,414.287235
2,2021-01-29,620.741659,202.839631
3,2021-02-01,1290.781783,537.023116
4,2021-02-02,1084.047278,460.445252


In [28]:
def compare_ans(DIR):
    answer = pd.read_csv(DIR)

    lunch_answer = np.array(answer.iloc[:,1])
    dinner_answer = np.array(answer.iloc[:,2])
    
    lunch_MAE = abs(predicts_Auto_lunch - lunch_answer).mean()
    dinner_MAE = abs(predicts_Auto_dinner - dinner_answer).mean()
    
    print("lunch_MAE : ", lunch_MAE)
    print("dinner_MAE : ", dinner_MAE)
    print("total_MAE : ", (lunch_MAE+dinner_MAE)/2)
    
    
compare_ans('../submission/20210701_lgbm_knfold_drop.csv')
compare_ans('../submission/20210715_pycaret_(2).csv')
compare_ans('../submission/20210715_lgbm_autoML.csv')

lunch_MAE :  19.060290110391904
dinner_MAE :  16.000110031227837
total_MAE :  17.53020007080987
lunch_MAE :  58.67831945287493
dinner_MAE :  37.069272300307965
total_MAE :  47.873795876591444
lunch_MAE :  14.955894937446146
dinner_MAE :  10.091844176192525
total_MAE :  12.523869556819335


In [24]:
import math

math.cos(math.pi*68.44/180)*69.5

25.539537282882215

# 저장

In [25]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_lgbm_autoML.csv', index =False)

오늘 날짜 : 20210716


In [26]:
answer = pd.read_csv('../submission/20210715_pycaret_(2).csv')

In [27]:
best_submission = pd.read_csv('../data/sample_submission.csv')
best_submission.iloc[:,1:] = submission.iloc[:,1:]*5/9 +  answer.iloc[:,1:]*4/9
best_submission.to_csv(f'../submission/{today}_lgbm_autoML_ensenble.csv', index =False)